In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
import tensorflow as tf

In [47]:
import tensorflow.keras as K

In [48]:
from tensorflow.keras import layers,models,Model

In [49]:
train_dir='/content/drive/MyDrive/plantdata/train'
test_dir='/content/drive/MyDrive/plantdata/test'
val_dir='/content/drive/MyDrive/plantdata/val'

In [64]:
img_size = (32,32)
batch_size = 16

train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="int",
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True
)
test_dataset=tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels="inferred",
    label_mode="int",
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True
)
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='int',
    shuffle=False
)



Found 3251 files belonging to 4 classes.
Found 697 files belonging to 4 classes.
Found 697 files belonging to 4 classes.


In [65]:
normalization_layer=tf.keras.layers.Rescaling(1./255)

In [66]:
train_dataset=train_dataset.map(lambda x,y:(normalization_layer(x),y))
test_dataset=test_dataset.map(lambda x,y:(normalization_layer(x),y))
val_dataset=val_dataset.map(lambda x,y:(normalization_layer(x),y))

In [68]:
base_model = tf.keras.applications.ResNet152(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model.layers:
  layer.trainable = False

In [54]:
x=layers.Flatten()(base_model.output)
x=layers.Dense(128, activation='relu')(x)
x=layers.Dropout(0.5)(x)
predictions=layers.Dense(4, activation='softmax')(x)

In [55]:
head_model = Model(inputs = base_model.input, outputs = predictions)
head_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [85]:
history=head_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
    )

Epoch 1/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 82s 403ms/step - accuracy: 0.9942 - loss: 0.0372 - val_accuracy: 0.8867 - val_loss: 1.0093
Epoch 2/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 141s 400ms/step - accuracy: 0.9858 - loss: 0.0907 - val_accuracy: 0.9957 - val_loss: 0.0292
Epoch 3/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 82s 404ms/step - accuracy: 0.9808 - loss: 0.0920 - val_accuracy: 0.9799 - val_loss: 0.0706
Epoch 4/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 82s 403ms/step - accuracy: 0.9889 - loss: 0.0382 - val_accuracy: 0.9785 - val_loss: 0.0682
Epoch 5/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 82s 401ms/step - accuracy: 0.9884 - loss: 0.0447 - val_accuracy: 0.9885 - val_loss: 0.0674
Epoch 6/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 82s 402ms/step - accuracy: 0.9962 - loss: 0.0167 - val_accuracy: 0.9570 - val_loss: 0.1717
Epoch 7/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 141s 400ms/step - accuracy: 0.9827 - loss: 0.0632 - val_accuracy: 0.9742 - val_loss: 0.0872
Epoch 8/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 82s 401ms/step - accuracy: 0.9797 - loss:

In [86]:
print(head_model.evaluate(test_dataset))

44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 116ms/step - accuracy: 0.9795 - loss: 0.0518
[0.06972457468509674, 0.9756097793579102]


In [87]:
print(head_model.evaluate(val_dataset))


44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 115ms/step - accuracy: 0.9848 - loss: 0.0720
[0.12302979826927185, 0.9713056087493896]


In [91]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Define the class names based on your directory structure
class_names = ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy']

def predict_image(image_path, model):
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize

    # Make prediction
    predictions = model.predict(img_array)
    predicted_class_idx = np.argmax(predictions, axis=-1)[0]
    predicted_class = class_names[predicted_class_idx]
    confidence = np.max(predictions)

    return predicted_class, confidence

# Example prediction
image_path = '2.JPG'  # Replace with the path to your image
predicted_class, confidence = predict_image(image_path, head_model)
print(f"Predicted Class: {predicted_class}, Confidence: {confidence:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Predicted Class: Apple___healthy, Confidence: 1.00


In [92]:
head_model.save('/content/drive/My Drive/plantdata/appletl.h5')